<a href="https://colab.research.google.com/github/Jeevesh8/arg_mining/blob/main/experiments/long_context_am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies

In [ ]:
%%capture
#if running on colab, install below 4
#!git clone https://github.com/Jeevesh8/arg_mining
#!pip install transformers
#!pip install seqeval datasets allennlp
#!pip install flax

#if connected to local runtime, run the next command too
#pip install bs4 tensorflow torch 



*   Update ``arg_mining/datasets/cmv_modes/configs.py`` as per your requirements, all experiments considered till now, set ``batch_size`` to 2, and all other variables with their default value.



In [1]:
#Run to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Load Metric

In [2]:
from datasets import load_metric
metric = load_metric('seqeval')

### Define & Load Tokenizer, Model, Dataset

In [3]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda', index=0)

In [5]:
model_version = 'allenai/longformer-base-4096'

In [6]:
%%capture
from transformers import LongformerTokenizer, AutoModel
tokenizer = LongformerTokenizer.from_pretrained(model_version)
transformer_model = AutoModel.from_pretrained(model_version).to(device)

In [7]:
%%capture
from arg_mining.datasets.cmv_modes import load_dataset, data_config

tokenizer.add_tokens(data_config["special_tokens"])

transformer_model.resize_token_embeddings(len(tokenizer))

def get_datasets():
    train_dataset, valid_dataset, test_dataset = load_dataset(tokenizer=tokenizer,
                                                              train_sz=80,
                                                              test_sz=20,)
    return train_dataset, valid_dataset, test_dataset

### Define layers for a Linear-Chain-CRF

In [8]:
import torch.nn as nn

from allennlp.modules.conditional_random_field import ConditionalRandomField as crf

ac_dict = data_config["arg_components"]

allowed_transitions =([(ac_dict["B-C"], ac_dict["I-C"]), 
                       (ac_dict["B-P"], ac_dict["I-P"])] + 
                      [(ac_dict["I-C"], ac_dict[ct]) 
                        for ct in ["I-C", "B-C", "B-P", "O"]] +
                      [(ac_dict["I-P"], ac_dict[ct]) 
                        for ct in ["I-P", "B-C", "B-P", "O"]] +
                      [(ac_dict["O"], ac_dict[ct]) 
                        for ct in ["O", "B-C", "B-P"]])
                    
linear_layer = nn.Linear(transformer_model.config.hidden_size,
                         len(ac_dict)).to(device)

crf_layer = crf(num_tags=len(ac_dict),
                constraints=allowed_transitions,
                include_start_end_transitions=False).to(device)

cross_entropy_layer = nn.CrossEntropyLoss(weight=torch.log(torch.tensor([3.8087, 54.6686, 3.4161, 45.3662, 2.4729], 
                                                                        device=device)), reduction='none')

### Global Attention Mask Utility for Longformer

In [9]:
import numpy as np
from threading import Lock
MUTEX = Lock()

def get_global_attention_mask(tokenized_threads: np.ndarray) -> np.ndarray:
    """Returns an attention mask, with 1 where there are [USER{i}] tokens and 
    0 elsewhere.
    """
    mask = np.zeros_like(tokenized_threads)
    for user_token in ["UNU"]+[f"[USER{i}]" for i in range(data_config["max_users"])]:
        MUTEX.acquire()
        try:
            user_token_id = tokenizer.encode(user_token)[1:-1]
        finally:
            MUTEX.release()
        mask = np.where(tokenized_threads==user_token_id, 1, mask)
    return np.array(mask, dtype=bool)

### Loss and Prediction Function

In [10]:
from typing import Tuple

In [11]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
            preds: bool=False, cross_entropy: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        preds:  If True, returns a List(of batch_size size) of Tuples of form 
                (tag_sequence, viterbi_score) where the tag_sequence is the 
                viterbi-decoded sequence, for the corresponding sample in the batch.
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, comp_type_labels, global_attention_mask = batch
    
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,
                               attention_mask=pad_mask,
                               global_attention_mask=global_attention_mask).last_hidden_state)
    
    if preds:
        return crf_layer.viterbi_tags(logits, pad_mask)
    
    log_likelihood = crf_layer(logits, comp_type_labels, pad_mask)
    
    if cross_entropy:
        logits = logits.reshape(-1, logits.shape[-1])
        
        pad_mask, comp_type_labels = pad_mask.reshape(-1), comp_type_labels.reshape(-1)
        
        ce_loss = torch.sum(pad_mask*cross_entropy_layer(logits, comp_type_labels))
        
        return ce_loss - log_likelihood

    return -log_likelihood

### Define optimizer

In [12]:
from itertools import chain

import torch.optim as optim

optimizer = optim.AdamW(params = chain(transformer_model.parameters(),
                                      linear_layer.parameters(),
                                      crf_layer.parameters()),
                       lr = 2e-5, 
                       weight_decay = 1e-7)

### Training And Evaluation Loops

In [13]:
def train(dataset):
    accumulate_over = 4
    
    optimizer.zero_grad()

    for i, (tokenized_threads, masked_threads, comp_type_labels, _ ) in enumerate(dataset):
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads), 
                                             device=device, dtype=torch.int32)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device, dtype=torch.long)

        loss = compute((tokenized_threads, 
                        comp_type_labels, 
                        global_attention_mask))/data_config["batch_size"]
        
        print("Loss: ", loss)
        loss.backward()
        
        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
    
    optimizer.step()

In [14]:
def evaluate(dataset, metric):
    
    int_to_labels = {v:k for k, v in ac_dict.items()}
    
    for tokenized_threads, masked_threads, comp_type_labels, _ in dataset:
        
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads), 
                                             device=device)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device)

        preds = compute((tokenized_threads,
                         comp_type_labels,
                         global_attention_mask),
                        preds=True)
        
        lengths = torch.sum(torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0), 
                            axis=-1)
        
        preds = [ [int_to_labels[pred] for pred in pred[0][:lengths[i]]]
                  for i, pred in enumerate(preds)
                ]
        
        refs = [ [int_to_labels[ref] for ref in labels[:lengths[i]]]
                 for i, labels in enumerate(comp_type_labels.cpu().tolist())
               ]
        
        metric.add_batch(predictions=preds, 
                         references=refs)
    
    print(metric.compute())

### Final Training

In [15]:
n_epochs = 25

In [16]:
for epoch in range(n_epochs):
    print(f"------------EPOCH {epoch+1}---------------")
    train_dataset, _, test_dataset = get_datasets()
    train(train_dataset)
    evaluate(test_dataset, metric)

------------EPOCH 1---------------


fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.
2021-08-25 05:51:50.835721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-25 05:51:50.836507: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loss:  tensor(3231.1399, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2643.7749, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3479.6191, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3222.1128, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2114.4832, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2084.6899, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2665.1609, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2929.5229, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1902.2341, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1914.3705, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2024.3479, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3285.8921, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2906.3728, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4192.4355, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3961.7925, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  ten

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(2539.8989, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2013.8823, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2727.1030, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2327.2808, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1638.6561, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1626.0693, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2028.9069, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2194.4111, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1555.0105, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1661.2297, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1714.0554, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2617.7202, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2382.6133, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3531.3945, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3391.4980, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  ten

...

Loss:  tensor(42.5628, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(50.3781, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(63.6691, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(31.5774, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(33.3112, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(27.2876, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(45.0443, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(361.5530, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(20.1241, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(21.8311, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(20.8862, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(40.3065, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(589.8832, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1472.4177, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1505.6975, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(44.8055, device='cud

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(37.4467, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(38.3426, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(46.4677, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(26.5091, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(29.2047, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(23.9625, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(37.5225, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(346.8550, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(18.5343, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(19.4218, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(22.9389, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(35.4801, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(581.9138, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1459.7927, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1484.4738, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(48.8235, device='cud

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(42.4605, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(35.4201, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(32.3064, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(24.6893, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(25.5334, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(22.1771, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(29.0076, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(337.2578, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15.5480, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14.4571, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(22.2739, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(32.0493, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(575.6185, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1452.4900, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1469.2528, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(40.6803, device='cud

In [17]:
for epoch in range(n_epochs-10):
    print(f"------------EPOCH {epoch+1+25}---------------")
    train_dataset, _, test_dataset = get_datasets()
    train(train_dataset)
    evaluate(test_dataset, metric)

------------EPOCH 26---------------


fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(46.7878, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(32.0810, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(32.0098, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(41.8475, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(29.3289, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(23.0600, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(30.7739, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(332.6900, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15.5055, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(20.6990, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(28.5847, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(42.5827, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(570.0729, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1446.2185, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1459.5818, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(34.1197, device='cud

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(32.5197, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(18.5947, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(21.8253, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(23.3939, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(23.8054, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(17.9277, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(19.9736, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(323.7236, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15.1145, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(21.8164, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(18.2888, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(40.4683, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(575.9846, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1467.7380, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1478.1648, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(52.3935, device='cud

...

Loss:  tensor(11.8181, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.5429, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.2648, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13.9540, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.1525, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.5541, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.4768, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(274.3262, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.6104, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.9968, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.8722, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14.1986, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(516.3518, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1374.9719, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1323.2546, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.9742, device='cuda:0', gr

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(7.0057, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.9521, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.7589, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.9982, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.6007, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.9460, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.0923, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(271.9830, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.2796, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.0970, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.5677, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13.3212, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(509.5109, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1373.2252, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1316.2278, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.2161, device='cuda:0', gra

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(5.9545, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.3894, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.3463, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.3966, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.2059, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.4314, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.7198, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(268.0307, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.0803, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.4525, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.2447, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.4803, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(506.0266, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1351.5155, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1296.5713, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.6786, device='cuda:0', grad